# Chromatographic Processes - Exercise

## Exercise 1: Combine nonbinding tracer with binding components

From the tutorial, combine the dextran pulse with the langmuir experiment.
Then, start modifying:
- Feed concentrations
- Adsorption parameters
- Transport parameters
- Flow Rates

## Exercise 2: Multiple injections

For some processes, multiple injections onto a column in sequence.
Take the previous example and create an inlet profile with three injections.
For this purpose, the `Simulator` can automatically run multiple simulations by setting `n_cycles`.

***Task:*** Try finding the best interval (cycle time) s.t. the column is used most efficiently.

## Example 3: Load wash elute with three components

Add a second protein component to the LWE example from the tutorial lesson.
Assume that all parameters are the same as the first protein, only change:
- adsorption rate: $0.3~m^{3}_{MP}m^{-3}_{SP}s^{-1}$
- characteristic charge: $5.0$